In [1]:
# # Run this notebook on Colab with T4 GPU

# import tensorflow as tf
# tf.config.list_physical_devices('GPU')

# from google.colab import drive
# drive.mount("/content/drive")

# DATA_FOLDER = "/content/drive/My Drive/Data/"

In [2]:
DATA_FOLDER = "./data/"

In [3]:
import numpy as np
from keras import regularizers
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import LSTM, Dense, Masking, Dropout, Bidirectional, Input, Concatenate
from keras.models import Model

from typing import Any

import h5py

import cProfile

In [4]:
FILENAME_TRAIN= f"{DATA_FOLDER}train_data_post.h5"
with h5py.File(FILENAME_TRAIN, 'r') as hf:
    X_train_masked_words = hf["masked_words"]
    X_train_guesses = hf["previous_guesses"]
    y_train = hf['next_guess_probs']

    assert X_train_masked_words.shape[0] == X_train_guesses.shape[0] == y_train.shape[0]
    LEN_TRAIN = y_train.shape[0]
    print(f"Number of training samples and labels: {LEN_TRAIN}")


FILENAME_VAL= f"{DATA_FOLDER}validation_data_post.h5"
with h5py.File(FILENAME_VAL, 'r') as hf:
    X_val_masked_words = hf["masked_words"]
    X_val_guesses = hf["previous_guesses"]
    y_val = hf['next_guess_probs']

    assert X_val_masked_words.shape[0] == X_val_guesses.shape[0] == y_val.shape[0]
    LEN_VAL = y_val.shape[0]
    print(f"Number of validation samples and labels: {LEN_VAL}")

Number of training samples and labels: 7665680
Number of validation samples and labels: 1912161


## Build the model

In [5]:
def generate_samples_and_labels(filename: str, batch_size: int):
    with h5py.File(filename, 'r') as hf:
        X_masked_words = hf['masked_words']
        X_guesses = hf['previous_guesses']
        y = hf['next_guess_probs']

        for i in range(0, y.shape[0], batch_size):
            batch_X_masked_words = X_masked_words[i:i + batch_size]
            batch_X_guesses = X_guesses[i:i + batch_size]
            batch_y = y[i:i + batch_size]

            yield (batch_X_masked_words, batch_X_guesses), batch_y

In [6]:
def build_hangman_model(max_word_len: int, alphabet_len: int):
    alphabet_len_with_mask = alphabet_len + 1

    # Define input layers for masked word and guessed letters
    hidden_word_input = Input(
        shape=(max_word_len, alphabet_len_with_mask), name="hidden_word_input"
    )
    guessed_letters_input = Input(shape=(alphabet_len,), name="guessed_letters_input")

    hidden_word_input_masked = Masking(mask_value=-1)(hidden_word_input)

    # LSTM layer to process the hidden word
    lstm_output = Bidirectional(LSTM(128))(hidden_word_input_masked)
    lstm_dropout = Dropout(0.1)(lstm_output)

    # Concatenate LSTM output with guessed letters
    combined_input = Concatenate()([lstm_dropout, guessed_letters_input])

    # Dense layer
    dense_output = Dense(
        64,
        activation="relu",
        kernel_regularizer=regularizers.l2(0.01)
    )(combined_input)
    dense_dropout = Dropout(0.1)(dense_output)

    # Output layer
    output = Dense(alphabet_len, activation="softmax", name="output")(dense_dropout)

    # Define model with inputs and output
    model = Model(inputs=[hidden_word_input, guessed_letters_input], outputs=output)

    return model

In [7]:
MAX_WORD_LEN = 30
ALPHABET_LEN = 26

# Build the model
model = build_hangman_model(MAX_WORD_LEN, ALPHABET_LEN)

# Load previous weights
# model_weights = f"{DATA_FOLDER}hangman_model.weights.h5"
# model.load_weights(model_weights)

model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ hidden_word_input   │ (None, 30, 27)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal           │ (None, 30, 27)    │          0 │ hidden_word_inpu… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ masking (Masking)   │ (None, 30, 27)    │          0 │ hidden_word_inpu… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ any (Any)           │ (None, 30)        │          0 │ not_equal[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional       │ (None, 256)       │    159,744 │ masking[0][0],    │
│ (Bidirectional)     │                   │            │ any[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 256)       │          0 │ bidirectional[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ guessed_letters_in… │ (None, 26)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 282)       │          0 │ dropout[0][0],    │
│ (Concatenate)       │                   │            │ guessed_letters_… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 64)        │     18,112 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 64)        │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output (Dense)      │ (None, 26)        │      1,690 │ dropout_1[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 179,546 (701.35 KB)

 Trainable params: 179,546 (701.35 KB)

 Non-trainable params: 0 (0.00 B)

In [8]:
# Train the model
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

early_stopping = EarlyStopping(monitor="val_loss", patience=2, verbose=1, restore_best_weights=True)
checkpoint_callback = ModelCheckpoint(
    filepath=f"{DATA_FOLDER}hangman_model.weights.h5",
    save_weights_only=True,
    save_best_only=True
)

epochs = 4
batch_size = 64

for _ in range(epochs):
    train_generator = generate_samples_and_labels(FILENAME_TRAIN, batch_size)
    val_generator = generate_samples_and_labels(FILENAME_VAL, batch_size)

    history = model.fit(
        train_generator,
        steps_per_epoch=LEN_TRAIN // batch_size,
        epochs=1,
        validation_data=val_generator,
        validation_steps=LEN_VAL // batch_size,
        callbacks=[early_stopping, checkpoint_callback],
    )

model.save(f"{DATA_FOLDER}hangman_model.keras")

## Predictions

In [8]:
# Load the model

hangman_model = build_hangman_model(MAX_WORD_LEN, ALPHABET_LEN)
weights = f"{DATA_FOLDER}hangman_model.weights.h5"
hangman_model.load_weights(weights)

In [9]:
# Necessary imports and data structures for making predictions

import collections
from keras.preprocessing.sequence import pad_sequences

from utils.samples_and_labels import (
    CHAR_TO_IDX,
    IDX_TO_CHAR,
    build_len_to_most_frequent_letters,
    build_len_to_words_trie,
    find_matching_words_in_trie,
    sample_from_masked_word_and_guesses,
)
from utils.trie import Trie


# Load the dictionary
dictionary_file_location = "./data/words_250000_train.txt"
text_file = open(dictionary_file_location, "r")
full_dictionary = text_file.read().splitlines()
text_file.close()

LEN_TO_MOST_FREQUENT_LETTERS = build_len_to_most_frequent_letters(full_dictionary)
LEN_TO_WORDS_TRIE = build_len_to_words_trie(full_dictionary)

In [10]:
# Function to predict the next letter

def get_next_letter_probs_with_model(masked_word: str, guesses: set[str], model: Any) -> np.ndarray:
    """Get the probabilities of the next letter given the masked word and guesses."""
    masked_word_encoding, guesses_encoding = sample_from_masked_word_and_guesses(masked_word, guesses)

    masked_word_sample = pad_sequences([masked_word_encoding], maxlen=MAX_WORD_LEN, padding='post', value=-1)
    guesses_sample = np.array([guesses_encoding])

    prediction_vector = model.predict([masked_word_sample, guesses_sample], verbose=0)[0]
    return prediction_vector


def get_next_letter_from_trie(masked_word: str, incorrect_guesses: set[str], len_to_words_trie: dict[int, Trie]) -> str:
    """Get the next letter using the trie. Check for similar length words in the trie that do not contain incorrect guesses."""
    all_matching_words = []
    for len_to_check in range(len(masked_word) - 1, len(masked_word) + 2):
        if len_to_check not in len_to_words_trie:
            continue
        pattern = "".join("_" for _ in range(len_to_check))
        matching_words = find_matching_words_in_trie(pattern, incorrect_guesses, len_to_words_trie)
        all_matching_words.extend(matching_words)

    matching_words_full_str = "".join(all_matching_words)
    counter = collections.Counter(matching_words_full_str).most_common()

    for char, _ in counter:
        if char not in incorrect_guesses and char not in masked_word:
            return char


def predict_next_letter(masked_word: str, guesses: set[str], model: Any) -> tuple[str, np.ndarray | None]:
    """Predict the next letter given the masked word and guesses using the given model"""
    correct_guesses = set(masked_word) - {"_"}
    incorrect_guesses = guesses - correct_guesses

    percentage_of_unknown = masked_word.count("_") / len(masked_word)

    # If there are less than 2 correct guesses, use the trie to predict the next letter
    # Check for same length words in the trie that do not contain incorrect guesses
    if len(correct_guesses) < 1 or percentage_of_unknown > 0.75 and len(correct_guesses) < 2:
        next_letter = get_next_letter_from_trie(masked_word, incorrect_guesses, LEN_TO_WORDS_TRIE)
        return next_letter, None

    # Predict the next letter using the model
    prediction_vector = get_next_letter_probs_with_model(masked_word, guesses, model)

    # Set probabilities of guessed letters to 0
    prediction_vector[[CHAR_TO_IDX[char] for char in guesses]] = 0

    if sum(prediction_vector) == 0:
        print("No valid predictions left. Guessing the most frequent letter")
        for char, _ in LEN_TO_MOST_FREQUENT_LETTERS[len(masked_word)]:
            if char not in guesses:
                return char, None

    predicted_letter = IDX_TO_CHAR[np.argmax(prediction_vector)]
    return predicted_letter, prediction_vector


## Play

In [11]:
import random
from sklearn.model_selection import train_test_split

words_train, words_val = train_test_split(full_dictionary, test_size=0.2, random_state=42)
print(f"Number of words_train: {len(words_train)}")
print(f"Number of words_val: {len(words_val)}")

Number of words_train: 181840
Number of words_val: 45460


In [12]:
# Test the model by playing hangman

# word_list = full_dictionary
# word_list = words_train
word_list = words_val

success_count = 0
total_games = 100

for i in range(total_games):
    word = random.choice(full_dictionary)
    print(f"GAME {i+1}: {word}")

    masked_word = ["_"] * len(word)
    guesses = set()
    incorrect_guesses = set()

    word_chars = set(word)

    while "_" in masked_word and len(incorrect_guesses) < 6:
        letter, _ = predict_next_letter(
            masked_word,
            guesses,
            hangman_model
        )
        guesses.add(letter)
        print(f"{letter}", end=": ")

        if letter in word_chars:
            for j, char in enumerate(word):
                if char == letter:
                    masked_word[j] = letter
        else:
            incorrect_guesses.add(letter)
        print("".join(masked_word))

    success = "_" not in masked_word
    print('SUCCESS' if success else 'FAILURE')

    if success:
        success_count += 1

print("Success rate:", success_count/total_games)

GAME 1: ensheathed
e: e___e___e_
r: e___e___e_
d: e___e___ed
t: e___e_t_ed
h: e__he_thed
a: e__heathed
s: e_sheathed
n: ensheathed
SUCCESS
GAME 2: jamaal
e: ______
a: _a_aa_
l: _a_aal
r: _a_aal
n: _a_aal
b: _a_aal
t: _a_aal
c: _a_aal
FAILURE
GAME 3: angka
e: _____
a: a___a
r: a___a
n: an__a
t: an__a
d: an__a
g: ang_a
i: ang_a
l: ang_a
FAILURE
GAME 4: fredia
e: __e___
a: __e__a
r: _re__a
s: _re__a
t: _re__a
n: _re__a
p: _re__a
c: _re__a
d: _red_a
i: _redia
o: _redia
FAILURE
GAME 5: possessival
e: ____e______
i: ____e__i___
n: ____e__i___
r: ____e__i___
t: ____e__i___
s: __ssessi___
o: _ossessi___
p: possessi___
v: possessiv__
a: possessiva_
l: possessival
SUCCESS
GAME 6: boisseau
e: _____e__
a: _____ea_
r: _____ea_
l: _____ea_
n: _____ea_
t: _____ea_
d: _____ea_
b: b____ea_
h: b____ea_
FAILURE
GAME 7: sorriness
e: ______e__
i: ____i_e__
n: ____ine__
s: s___iness
l: s___iness
r: s_rriness
u: s_rriness
a: s_rriness
o: sorriness
SUCCESS
GAME 8: whitishflowered
e: ___________e_e_
i: __i_i__